In [28]:
import numpy as np
import pandas as pd

import os # used for navigating to image path
import imageio # used for writing images

from sklearn.preprocessing import LabelEncoder

import keras

#  Keras preprocessing
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image_dataset_from_directory

# Keras modeling
from keras.models import Sequential
from keras.layers import  Lambda , Dense, Flatten, Dropout, Conv2D, MaxPool2D
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization, Convolution2D , MaxPooling2D
from keras import layers

In [29]:

train = pd.read_csv('W21P2_train.csv', error_bad_lines=False,  encoding='latin1')
test = pd.read_csv('W21P2_test.csv', error_bad_lines=False,  encoding='latin1')

In [30]:
train

,interest_level,bathrooms,bedrooms,latitude,longitude,price,street_address,Doorman,Elevator,Laundry.in.Building,...,High.speed.Internet,Tenant.Lounge,high.ceilings,Private.backyard,Private.outdoor.space,Gym.In.Building,Decorative.Fireplace,Central.AC,Sundeck,Yoga.Classes
0,medium,2.0,4,40.7182,-74.0148,5354,41 River Terrace,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,low,1.0,2,40.7231,-73.9938,4695,232 Elizabeth Street,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,medium,1.0,1,40.7463,-73.9132,1850,39-89 51st St,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,medium,1.0,2,40.7349,-73.9818,3030,324 E 19 Street,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,high,1.0,1,40.7031,-73.8119,1500,88-40 144th St,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8825,low,1.0,1,40.7769,-73.9494,2050,E. 86th & 1st Ave.,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8826,high,1.0,0,40.8665,-73.9186,1400,521 Isham Street,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8827,low,1.0,0,40.7047,-74.0076,2795,95 Wall Street,1,1,1,...,0,0,0,0,0,0,0,0,0,0
8828,low,0.0,1,40.7730,-73.9509,2100,422 E 81 St,0,0,0,...,0,0,0,0,0,0,0,0,0,0


delete 2541, 4685, 6174, 7707th row for text analysis

In [31]:
train.drop([2540, 4684, 6173, 7706], inplace=True)

In [32]:
train_text = train['street_address']
test_text = test['street_address']
train_tags = train['interest_level']

In [33]:
train_text

0           41 River Terrace
1       232 Elizabeth Street
2              39-89 51st St
3            324 E 19 Street
4             88-40 144th St
                ...         
8825     E. 86th & 1st Ave. 
8826        521 Isham Street
8827          95 Wall Street
8828             422 E 81 St
8829          341 W 45th St.
Name: street_address, Length: 8826, dtype: object

In [34]:
from keras.preprocessing import text, sequence

max_words = 10000
train_text = train_text.astype(str)
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_text)
x_train = tokenize.texts_to_matrix(train_text)
test_text = test_text.astype(str)
x_test = tokenize.texts_to_matrix(test_text)

In [37]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
pca.fit(x_train)
print(pca.explained_variance_ratio_)

[0.12451118 0.05536443 0.04736125 0.04019613 0.02540724]


In [38]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca.fit(x_train)
print(pca.explained_variance_ratio_)

[0.12451118]


In [39]:
pca_train = pca.transform(x_train)
pca_test = pca.transform(x_test)

In [41]:
pd.DataFrame(pca_train).to_csv('text_analysis_train.csv', index = False)
pd.DataFrame(pca_test).to_csv('text_analysis_test.csv', index = False)